In [1]:
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import keras
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import Flatten, Dense, Input
from tensorflow.keras.models import Model


In [2]:
train_dir = "/Users/jarvis/pymycod/SIH/final-datasetchanger/train/"
test_dir = "/Users/jarvis/pymycod/SIH/final-datasetchanger/test/"

In [3]:
train_datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.2, 
        height_shift_range=0.2, 
        zoom_range=0.2, 
        horizontal_flip=True, 
        brightness_range=[0.2,1.2]) 
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256,256),
    batch_size=128,
    class_mode='categorical'
)

Found 1752 images belonging to 7 classes.


In [4]:
test_datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.2, 
        height_shift_range=0.2, 
        zoom_range=0.2, 
        horizontal_flip=True, 
        brightness_range=[0.2,1.2]) 
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(256,256),
    batch_size=128,
    class_mode='categorical'
)

Found 1146 images belonging to 7 classes.


In [5]:
vgg_model = VGG19(weights = 'imagenet',  include_top = False, input_shape = (256, 256, 3)) 
for models in vgg_model.layers:
  models.trainable= False

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-09-12 09:53:33.465238: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-09-12 09:53:33.465369: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [6]:

model = keras.Sequential()
for l in vgg_model.layers:
  model.add(l)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(200, activation='relu'))
model.add(keras.layers.Dense(180, activation='relu'))
model.add(keras.layers.Dense(6, activation='softmax'))


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 64, 64, 256)       2

In [8]:
# mdl = VGG19(
#     include_top=False,
#     weights="imagenet"
# )
# inputs = Input(shape=(150, 150, 3))
# mdl.trainable = False
# x = mdl(inputs)
# x = Flatten()(x)
# x = Dense(200, activation='relu')(x)
# outputs = Dense(14, activation='softmax')(x)
# model3 = Model(inputs, outputs)
# opt = tf.keras.optimizers.Adam(lr=0.001)
# model3.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
# model3.fit_generator(train_generator,epochs = 20,verbose = 2,validation_data=test_generator)
# tf.keras.models.save_model(model3,'./modelvgg.h5')

In [9]:
model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.Adam(lr=0.001),
                  metrics=['acc'])
from keras.callbacks import ModelCheckpoint
model_path = "/Users/jarvis/pymycod/SIH/lr_model.h5"
checkpoint = ModelCheckpoint(model_path,monitor="val_acc",verbose=1,save_best_only=True,mode=max)
callbacks_list=[checkpoint]
history = model.fit(train_generator,epochs=25,verbose=2,batch_size=128,validation_data=test_generator,callbacks=callbacks_list)

/opt/anaconda3/envs/tensor/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/25


2023-09-12 09:53:37.139588: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-09-12 09:53:37.670750: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2023-09-12 09:54:44.250839: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.



Epoch 1: val_acc improved from -inf to 0.29668, saving model to /Users/jarvis/pymycod/SIH/lr_model.h5
14/14 - 107s - loss: 3.5603 - acc: 0.3002 - val_loss: 2.7883 - val_acc: 0.2967 - 107s/epoch - 8s/step
Epoch 2/25

Epoch 2: val_acc improved from 0.29668 to 0.33857, saving model to /Users/jarvis/pymycod/SIH/lr_model.h5
14/14 - 109s - loss: 1.8841 - acc: 0.3790 - val_loss: 2.1852 - val_acc: 0.3386 - 109s/epoch - 8s/step
Epoch 3/25


KeyboardInterrupt: 

: 